In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
d = pd.read_csv('../data/AANB-Data-Final-Revised20210414.csv', encoding = "ISO-8859-1")#, encoding='unicode_escape')
d

,ID,oldID,Headword,forename,surname,gender,Free Born in North America,Free before 13th Amendment,Ever Enslaved,How was freedom attained,...,altDeathPlaceState,Death Place Country.1,Occupation #1,Occupation #2,Occupation #3,Occupation #4,Occupation #5,Occupation #6,Occupation #7,Occupation #8
0,36020.0,/opr/t0001/e1839,aaron,Aaron,,M,NaN,Yes,Enslaved,"Ran away to Lancaster, Ohio at the age of 29 o...",...,NaN,NaN,Slave,Fugitive Slave,Slave Narrative Author,NaN,NaN,NaN,NaN,NaN
1,34829.0,/opr/t0001/e0641,abraham,Abraham,,M,NaN,Yes,Enslaved,Fled enslavement and made his way to Florida,...,NaN,NaN,Slave,American Indian Leader,Interpreter,Fugitive Slave,NaN,NaN,NaN,NaN
2,35438.0,/opr/t0001/e1253,aliceofdunk'sferry,Alice of Dunk's Ferry,,F,NaN,No,Enslaved,Enslaved her entire life.,...,NaN,United States,Slave,Orator,NaN,NaN,NaN,NaN,NaN,NaN
3,35032.0,/opr/t0001/e0845,billy,Billy,,M,NaN,No,Enslaved,No evidence that he was freed,...,NaN,NaN,Slave,Alleged Traitor,NaN,NaN,NaN,NaN,NaN,NaN
4,39700.0,/opr/t0001/e5936,blackcaesar,Black Caesar,,M,NaN,Yes,Enslaved,Escaped from the slave ship on which he was be...,...,NaN,United States,Slave,Pirate,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,34295.0,/opr/t0001/e0107,cesar,Cesar,NaN,M,NaN,Yes,Enslaved,Requested his freedom and 100 pounds per year ...,...,NaN,United States,Slave,Patent Medicine Maker,Physician,NaN,NaN,NaN,NaN,NaN
1300,34375.0,/opr/t0001/e0187,esteban,Esteban,NaN,M,NaN,Yes,Enslaved,In 1534 he and three Spaniards escaped from th...,...,NaN,United States,Slave,Explorer,NaN,NaN,NaN,NaN,NaN,NaN
1301,34621.0,/opr/t0001/e0433,onesimus,Onesimus,NaN,M,NaN,Yes,Enslaved,He purchased his freedom from his owner and pr...,...,NaN,United States,Slave,Medical Scientist,NaN,NaN,NaN,NaN,NaN,NaN
1302,34722.0,/opr/t0001/e0534,stagolee,Stagolee,NaN,M,NaN,NaN,NaN,NaN,...,NaN,United States,Murderer,Folk Hero,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = d.copy()

religion = pd.read_excel('../data/religion_mappings.xlsx')
df = df.merge(religion[['oldID', 'religion_clean']], on='oldID', how='left')
del([religion])

# Remove unnecessary columns
df.drop(['Headword'], axis=1, inplace=True)

# Remove whitespace from col name
df.columns = [col.strip() for col in df.columns]

# Create boolean Ever Enslaved
df['ever_enslaved_bool'] = np.where(df['Ever Enslaved'].fillna('').str.lower().str.contains('enslaved'), "Yes", "No")

# Replace redundant labels
free_before_map = {
    'No (?)': 'No',
    'Yes (?)': 'Yes',
    'Unknown': 'Unknown',
    'Status changed during the Civil War': 'Yes',
    'NaN': 'Unknown',
    'Status Uncertain: Possibly freed by service in the Union Army': 'Unknown',
    'Briefly free, but reenslaved': 'No',
    'Unclear': 'Unknown',
    'yes': 'Yes',
    'Not formally.': 'No',
    'Unclear if she was ever enslaved.': 'Unknown',
    'Freed just before the 13th Amendment': 'Yes'
}
df['free_before_13_bool'] = df['Free before 13th Amendment'].replace(free_before_map)

# Remove extra whitespace
df['Free Born in North America'] = df['Free Born in North America'].str.strip()

# Create Free Born in North America boolean
df['Free Born in North America'].replace({'NaN': 'Unknown', '': 'Unknown'}, regex=True, inplace=True)
df['free_born_north_america_bool'] = np.where(
    df['Free Born in North America'].str.lower().str.contains('freeborn'),
    'Yes',
    np.where(df['Free Born in North America'] != 'Unknown', 'No', 'Unknown')
)

# Remove duplicate based on string case so all values unique
df['African born'] = df['African born'].str.capitalize()

# Remove duplicate based on strings meaning the same thing so all values unique
rebels_replace_map = {
    '  ': ' ',
    '\(.*\?\)': '',
    'N/A': 'Unknown',
    '^[N|n]o [E|e]vidence of+ [R|r]unaway\.?\s?$': 'No evidence of runaway',  # Catches most of these but some wonky binary chars
    'Unclear\.$': 'Unclear',
    'Fugitive Slave\.$': 'Fugitive Slave',
    '\s$': 'Unknown',
    '^nan$': 'Unknown'
}
df['Runaways and rebels'] = df['Runaways and rebels'].fillna('Unknown')
df['Runaways and rebels'] = df['Runaways and rebels'].str.strip()
df['Runaways and rebels'] = df['Runaways and rebels'].replace(rebels_replace_map, regex=True).str.strip()
df['Runaways and rebels'].replace('No evidence of runaway', 'No evidence of runaway', inplace=True) # Strange Characters not detected in regex
df['Runaways and rebels'].replace('No Evidence of Runaway', 'No evidence of runaway', inplace=True) # Strange Characters not detected in regex
df['Runaways and rebels'].replace('Probably a runaway', 'Probably a runaway', inplace=True) # Strange Characters not detected in regex
df['Runaways and rebels'].replace('', 'Unknown', inplace=True) # Strange Characters not detected in regex

# Runaways and rebels boolean
df['runaways_and_rebels_bool'] = np.where(
    (df['Runaways and rebels'].str.lower().str.contains('no evidence of runaway') | ((df['Runaways and rebels'].str.lower() == 'no') | (df['Runaways and rebels'].str.lower() == 'no.')) | df['Runaways and rebels'].str.lower().str.contains('did not')),
    'No',
    np.where(
        ((df['Runaways and rebels'].str.lower().str.contains('unknown')) & (~df['Runaways and rebels'].str.lower().str.contains('no evidence')) | (df['Runaways and rebels'].str.lower() == 'unclear') | df['Runaways and rebels'].str.lower().str.contains('no reference')),
        'Unknown',
        'Yes'
    )
)

# Education and Literacy
df['Education/literacy'] = df['Education/literacy'].fillna('Unknown').str.strip()
education_replace_map = {
    'Illiterate\.$': 'Illiterate',
    # 'IlLiterate\.?': 'Illiterate',
    '[L|l]iterate.$': 'Literate',
    'literate\.?$': 'Literate', # Strange characters
    'Unknown.$': 'Unknown',
    'Taught herself to read.$': 'Literate. Taught herself to read.',
    '__$': 'Unknown',
    '': 'Unknown',
    'No evidence of literacy$': 'No evidence of literacy.',
    'Literate. Learned to read and write while still enslaved$': 'Literate. Learned to read and write while still enslaved.'
}
df['Education/literacy'] = df['Education/literacy'].replace(education_replace_map, regex=True).str.strip()
df['Education/literacy'].replace('IlLiterate', 'Illiterate', inplace=True)
df['Education/literacy'].replace('Illiterate', 'Illiterate', inplace=True)
df['Education/literacy'].replace('literate. Enrolled at Atlanta University aged 17 in 1870. Published a book (posthumously). The House of Bondage; or, Charlotte Brooks and Other Slaves. (1890)', 'Literate. Enrolled at Atlanta University aged 17 in 1870. Published a book (posthumously). The house of Bondage; or, Charlotte Brooks and Other Slaves. (1890)', inplace=True)

df['education_prefix'] = np.where(
    len(df['Education/literacy'].str.split('.').str[:]) > 1,
    df['Education/literacy'].str.split('.').str[0],
    'None'
)

ed_unknown = ['Unknown', 'Unclear', 'No evidence of runaway', 'The details of his education are not stated', 'Educational details are unknown', 'Dictated an autobiography', 'Formal schooling is unknown', 'Details unknown', 'It is unclear if he became literate', 'Details on education are not listed', 'May have been literate', 'One of the Marie Laveaus could at least sign her name', 'Self-educated after acquiring a spelling book', 'Her educational background is unknown, but she apparently had a well-marked Bible', 'The details of his education are not listed', 'Details of his education are unlisted', 'There is no mention of formal education, but Ward briefly served in the Louisiana legislature', 'Little is known about her education', 'Apparently able to write her own name', 'Perhaps literate', 'Extensively trained in singing', 'Illiterate while enslaved', 'Phillis Wheatley described him as a poet', 'Given music training', 'By 1877 he was able to sign his name', 'Unknown (likely ilLiterate', 'Apprenticed in cabinetry and tutored by a retired actor', 'The details of his education are unknown', 'No direct mention of literacy', 'Few details of his formal education are known', 'He and his siblings left school early to pursue work', 'Not stated', 'Played school with a white boy, taught in a Quaker-led afternoon slave school, and studied secretly with a white boy', "Educational background is unclear, but he published articles in Freedom's Journal and supported education", 'Jemmy was possibly the same person as Cato, a literate leader in the Stono uprising', 'Learned to read by a child in the family that owned him', 'Skilled at arithmetic', "His overseer's wife taught him to spell", 'His account was recorded in 1857 or 1858', 'No evidence of slavery', 'Briefly attended school in Oxford, Ohio']

ed_no = ['No evidence of literacy', 'Illiterate', 'No formal education', 'Could read but not write', 'Semiliterate', 'Little (no?) formal education', 'Illiterate, but memorized the scripture and songs necessary for his preaching', 'Attended public school but was expelled once his teachers learned that he was black', 'No evidence of literacy, though his son could read', 'Illiterate, but able to sign his name', 'Apparently never educated', 'Reportedly illiterate', 'Never attended school', 'Likely illiterate, but her husband (Gabriel Prosser) could read', 'Apparently had a limited education', 'No evidence of literacy, although a white apprentice was taught to learn to read while living with Fortune and her husband', 'Could read Arabic but could not write it', 'No formal education, but received instruction in piano', 'Semi-literate', 'Attended various Sunday schools and the school of Enoch Ambush, but was unable to write his name upon finishing his studies', 'No evidence of literacy, although the details of his schooling and photography training are unknown', 'Likely ilLiterate', 'No evidence of schooling', 'No evidence of literacy, although while enslaved he attended daily Bible study', 'Illiterate, but helped found several schools', 'Functionally illiterate', 'No direct evidence of literacy, though he was a leader in many organizations and fulfilled ministerial duties', 'Probably illiterate', 'Believed that she received no formal education', 'No evidence of education', 'Never attended school or wrote a letter', 'Illiterate at the time of his marriage', 'Apparently ilLiterate']

ed_yes = ['Literate', 'Apparently literate', 'Presumably literate', 'Likely literate', 'Possibly literate', 'Literate in Arabic', 'Received an elementary education while still enslaved and became a bookkeeper', 'Studied hard to become literate', 'In 1879, she completed a sixteen-month diploma program in nursing at the New England Hospital for Women and Children in Boston', 'Graduated in 1877 with a BA from the Augusta Institute (a forerunner of Morehouse College)', 'Attended school near the plantation where his family continued to work after Emancipation', 'Attended Sabbath school', 'Literate He underwent koranic and Arabic studies', 'Completed several months of schooling', 'His early education came from Bible study and catechism', 'Enrolled at the College of the Holy Cross in Worcester, Massachusetts, in 1850', "Educated in Freedmen's Bureau schools in Pine Bluff, Arkansas", 'After the Civil War he completed at least a year of public school in Pennsylvania towns', 'Educated in D', "Gained a basic education by following the lessons of his owner's son", 'After the Civil War ended, he briefly attended common schools', 'Graduated in 1870 from Brockport Collegiate Institute', 'Basic literacy', 'Literate and fluent in several languages', 'Educated at Suffield Institute in Connecticut', 'Received a basic education in a Quaker-run school', 'Both of his parents were literate and they probably arranged tutors for William and his siblings', 'Literate in his native language of Kanouri and in Arabic', 'Attended public schools in Salem until forced out due to her race', 'Attended the Institution for the Education of Colored Youth in Washington, D', 'Attended school in Selma, OH', 'Received a basic education', 'Educated by private tutors', 'Literate, likely self-educated', 'Attended school in Providence, RI', 'Graduated from Hampton Institute in Virginia in 1884', 'Taught to read and write by his white grandmother', 'Probably Literate', 'Literate; a public school teacher', 'Attended school until age 12 in North Branford, where the other pupils were white', 'Received a basic British-style education in Nassau in the Bahamas', 'Educated in England', 'Literate by 1870, though no formal education', 'Attended school from age 4 to 10', 'Educated at the Quaker School for Negroes in Philadelphia', "Tutored with his master's son", 'Literate and numerate', 'Likely Literate', 'Reportedly gained basic literacy while still enslaved', "Attended a Freedman's Bureau school", 'Reportedly went to school in France', 'Graduated from Hampton Institute in Virginia in 1878', "Attended Howard University's law department starting in 1883", "Attended a school organized by the Freedman's Bureau; this was likely her only formal education", 'Educated at the Wylie Street School for Colored Youth in Pittsburgh, PA', 'Studied at Ockham School in England', 'attended the Preparatory Department of Howard University, then the College Department (class of 1881 valedictorian)', 'Attended a school, received some tutoring, and then attended a Virginia public school', 'Apparently Literate', "Studied at rural schools and then at Alabama's Talladega College"]

df['education_literacy_bool'] = np.where(
    df['education_prefix'].isin(ed_unknown),
    'Unknown',
    np.where(
        df['education_prefix'].isin(ed_no),
        'No',
        np.where(
            df['education_prefix'].isin(ed_yes),
            'Yes',
            'Unknown'
        )
    )
)

df.drop('education_prefix', axis=1, inplace=True)


# Clean birth and death dates
NAN_YEAR = 9999
df['birthMonthClean'] = df['birthMonth'].fillna(1).astype(str).str.replace('\D+', '', regex=True)
df['birthDayClean'] = df['birthDay'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(int)
df['birthDayClean'] = np.where(
    df['birthDayClean'] < 1,
    1,
    df['birthDayClean']
)
df['birthYearClean'] = df['birthYear'].str.findall('\d{4}').explode().fillna(NAN_YEAR).astype(int)
df['altBirthYearClean'] = df['altBirthYear'].str.findall('\d{4}').explode().fillna(NAN_YEAR).astype(int)
df['deathYearClean'] = df['deathYear'].str.findall('\d{4}').explode().fillna(NAN_YEAR).astype(int)
df['altDeathYearClean'] = df['altDeathYear'].str.findall('\d{4}').explode().fillna(NAN_YEAR).astype(int)

df['dob'] = (df['birthMonthClean'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(str) + '/' + df['birthDayClean'].astype(str) + '/' + df['birthYearClean'].astype(str)).replace(f'1/1/{NAN_YEAR}', np.nan)
df['dod'] = (df['deathMonth'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(str) + '/' + df['deathDay'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(str) + '/' + df['deathYearClean'].astype(str)).replace(f'1/1/{NAN_YEAR}', np.nan)
df['alt_dob'] = (df['altBirthMonth'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(int).astype(str) + '/' + df['altBirthDay'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(int).astype(str) + '/' + df['altBirthYearClean'].astype(str)).replace(f'1/1/{NAN_YEAR}', np.nan)
df['alt_dod'] = (df['altDeathMonth'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(str) + '/' + df['altDeathDay'].replace(r'^\s*$', np.nan, regex=True).fillna(1).astype(str) + '/' + df['altDeathYearClean'].astype(str)).replace(f'1/1/{NAN_YEAR}', np.nan)

# Age at death
df['age_at_death'] = (pd.to_datetime(df[(df['birthYearClean'] != 9999) & (df['birthYearClean'] >= 1677)]['dod']) - pd.to_datetime(df[(df['birthYearClean'] != 9999) & (df['birthYearClean'] >= 1677)]['dob'])) / np.timedelta64(1, 'Y')

# Gender
df['gender'] = df['gender'].str.upper().replace('M, F.', np.nan)

# Full name
df['full_name'] = (df['forename'].fillna('').astype(str).str.strip() + ' ' + df['surname'].fillna('').astype(str).str.strip() + ' ' + df['Suffix'].fillna('').astype(str).str.strip()).str.strip()

df

,ID,oldID,forename,surname,gender,Free Born in North America,Free before 13th Amendment,Ever Enslaved,How was freedom attained,Other/Uncertain Status,...,birthYearClean,altBirthYearClean,deathYearClean,altDeathYearClean,dob,dod,alt_dob,alt_dod,age_at_death,full_name
0,36020.0,/opr/t0001/e1839,Aaron,,M,NaN,Yes,Enslaved,"Ran away to Lancaster, Ohio at the age of 29 o...",NaN,...,1845,9999,9999,9999,1/1/1845,NaN,NaN,NaN,NaN,Aaron
1,34829.0,/opr/t0001/e0641,Abraham,,M,NaN,Yes,Enslaved,Fled enslavement and made his way to Florida,NaN,...,1835,9999,9999,9999,1/1/1835,NaN,NaN,NaN,NaN,Abraham
2,35438.0,/opr/t0001/e1253,Alice of Dunk's Ferry,,F,NaN,No,Enslaved,Enslaved her entire life.,NaN,...,1686,9999,1802,9999,1/1/1686,1/1/1802,NaN,NaN,115.996906,Alice of Dunk's Ferry
3,35032.0,/opr/t0001/e0845,Billy,,M,NaN,No,Enslaved,No evidence that he was freed,NaN,...,1781,9999,9999,9999,1/1/1781,NaN,NaN,NaN,NaN,Billy
4,39700.0,/opr/t0001/e5936,Black Caesar,,M,NaN,Yes,Enslaved,Escaped from the slave ship on which he was be...,NaN,...,9999,9999,1718,9999,NaN,1/1/1718,NaN,NaN,NaN,Black Caesar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,34295.0,/opr/t0001/e0107,Cesar,NaN,M,NaN,Yes,Enslaved,Requested his freedom and 100 pounds per year ...,"Unknown if he was born in Africa, the Caribbea...",...,1682,9999,9999,9999,1/1/1682,NaN,NaN,NaN,NaN,Cesar
1304,34375.0,/opr/t0001/e0187,Esteban,NaN,M,NaN,Yes,Enslaved,In 1534 he and three Spaniards escaped from th...,NaN,...,1500,9999,1539,9999,1/1/1500,5/1/1539,NaN,NaN,NaN,Esteban
1305,34621.0,/opr/t0001/e0433,Onesimus,NaN,M,NaN,Yes,Enslaved,He purchased his freedom from his owner and pr...,Probably born in Africa,...,1706,9999,1717,9999,1/1/1706,1/1/1717,NaN,NaN,11.000910,Onesimus
1306,34722.0,/opr/t0001/e0534,Stagolee,NaN,M,NaN,NaN,NaN,NaN,Unknown Status at Birth. Born Texas 1865,...,1865,9999,1912,9999,3/16/1865,3/11/1912,NaN,NaN,46.985222,Stagolee


In [4]:
df.to_pickle('../data/aanb_data_clean.pkl')